In [1]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json
from glob import glob
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

import timm


In [2]:
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize
    ])    

    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    return transf(img).unsqueeze(0)

In [ ]:
ckpt_fn = "models/orig_20220627-235055-seresnext50_32x4d-224/last.pth.tar"
im_path = "data/orig_o"

fl_dict = {}
for idx, fl in enumerate(glob(f"{im_path}/*")):
    bn = os.path.basename(fl)
    fl_dict[bn] = idx

In [3]:
ckpt_fn = "models/planted_20220627-235814-seresnext50_32x4d-224/last.pth.tar"
im_path = "data/pl_o"
out_path = "pl_pl"

fl_dict = {}
for idx, fl in enumerate(glob(f"{im_path}/*")):
    bn = os.path.basename(fl)
    fl_dict[bn] = idx

In [10]:
ckpt_fn = "models/bl_20220703-220635-resnet50-224/last.pth.tar"
im_path = "data/bl"
out_path = "bl_bl"

fl_dict = {}
for idx, fl in enumerate(glob(f"{im_path}/*")):
    bn = os.path.basename(fl)
    fl_dict[bn] = idx

In [9]:
model = timm.create_model(
        'resnet50',
        num_classes=300,
        in_chans=3,
        pretrained=True,
        checkpoint_path=ckpt_fn)

target_layers = model.layer4

images = glob(f"{im_path}/**/*.*", recursive=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50_ram-a26f946b.pth" to /Users/dax/.cache/torch/hub/checkpoints/resnet50_ram-a26f946b.pth


In [12]:
cam = GradCAM(model=model, target_layers=target_layers)
eig = EigenCAM(model=model, target_layers=target_layers)
def process(im_fn, fl_dict):
    bn = os.path.basename(im_fn)
    cat = fl_dict[im_fn.split("/")[-2]]
    
    img = Image.open(im_fn)
    img_t = get_input_tensors(img)
    test_img = np.array(img.resize((224, 224)), dtype='float32')
    test_img /= 255

    targets = [ClassifierOutputTarget(cat)]

    grayscale_cam = cam(input_tensor=img_t, targets=targets)
    grayscale_cam = grayscale_cam[0, :]

    visualization = show_cam_on_image(test_img, grayscale_cam, use_rgb=True)
    plt.imshow(visualization)
    try:
        plt.savefig(f"outs/{out_path}/gradcam/{bn}")
    except FileNotFoundError:
        os.makedirs(f"outs/{out_path}/gradcam/")
    plt.clf()
    
    grayscale_eig = eig(input_tensor=img_t, targets=targets)
    grayscale_eig = grayscale_eig[0, :]

    visualization = show_cam_on_image(test_img, grayscale_eig, use_rgb=True)
    plt.imshow(visualization)
    try:
        plt.savefig(f"outs/{out_path}/eigencam/{bn}")
    except FileNotFoundError:
        os.makedirs(f"outs/{out_path}/eigencam/")
    plt.clf()
    
Parallel(n_jobs=8)(delayed(process)(im_fn, fl_dict) for im_fn in tqdm(images, total=len(list(images))))

  0%|          | 0/4230 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [6]:
abl = AblationCAM(model=model, target_layers=target_layers)
for im_fn in tqdm(images, total=len(list(images))):
    bn = os.path.basename(im_fn)
    cat = fl_dict[im_fn.split("/")[-2]]
    
    img = Image.open(im_fn)
    img_t = get_input_tensors(img)
    test_img = np.array(img.resize((224, 224)), dtype='float32')
    test_img /= 255

    targets = [ClassifierOutputTarget(cat)]
    
    grayscale_abl = abl(input_tensor=img_t, targets=targets)
    grayscale_abl = grayscale_abl[0, :]

    visualization = show_cam_on_image(test_img, grayscale_abl, use_rgb=True)
    plt.imshow(visualization)
    try:
        plt.savefig(f"outs/{out_path}/ablationcam/{bn}")
    except FileNotFoundError:
        os.makedirs(f"outs/{out_path}/ablationcam/")
    plt.clf()

  0%|          | 0/7859 [00:00<?, ?it/s]


100%|███████████████████████████████████████████| 64/64 [05:36<00:00,  5.26s/it]

100%|███████████████████████████████████████████| 64/64 [05:51<00:00,  5.49s/it]

 98%|██████████████████████████████████████████▎| 63/64 [05:45<00:05,  5.20s/it]


100%|███████████████████████████████████████████| 64/64 [05:50<00:00,  5.48s/it]

100%|███████████████████████████████████████████| 64/64 [05:14<00:00,  4.91s/it]

100%|███████████████████████████████████████████| 64/64 [05:41<00:00,  5.33s/it]

 97%|█████████████████████████████████████████▋ | 62/64 [05:36<00:10,  5.35s/it]


100%|███████████████████████████████████████████| 64/64 [05:47<00:00,  5.43s/it]

100%|███████████████████████████████████████████| 64/64 [05:44<00:00,  5.39s/it]

100%|███████████████████████████████████████████| 64/64 [05:45<00:00,  5.39s/it]

 95%|████████████████████████████████████████▉  | 61/64 [05:29<00:16,  5.48s/it]


100%|███████████████████████████████████████████| 64/64 [05:45<00:00,  5.40s/it]

100%|███████████████████████████████████████████| 64/64 [05:39<00:00,  5.30s/it]

100%|███████████████████████████████████████████| 64/64 [05:56<00:00,  5.56s/it]

 94%|████████████████████████████████████████▎  | 60/64 [05:32<00:22,  5.53s/it]


...

100%|███████████████████████████████████████████| 64/64 [05:52<00:00,  5.50s/it]

100%|███████████████████████████████████████████| 64/64 [05:38<00:00,  5.29s/it]

100%|███████████████████████████████████████████| 64/64 [05:53<00:00,  5.53s/it]

 84%|████████████████████████████████████▎      | 54/64 [04:58<00:55,  5.52s/it]


100%|███████████████████████████████████████████| 64/64 [05:54<00:00,  5.53s/it]

100%|███████████████████████████████████████████| 64/64 [05:39<00:00,  5.30s/it]

100%|███████████████████████████████████████████| 64/64 [05:53<00:00,  5.52s/it]

 83%|███████████████████████████████████▌       | 53/64 [04:53<01:00,  5.52s/it]


100%|███████████████████████████████████████████| 64/64 [05:54<00:00,  5.53s/it]

100%|███████████████████████████████████████████| 64/64 [05:40<00:00,  5.32s/it]

100%|███████████████████████████████████████████| 64/64 [05:54<00:00,  5.54s/it]

 81%|██████████████████████████████████▉        | 52/64 [04:48<01:06,  5.54s/it]


100%|███████████████████████████████████████████| 64/64 [05:54<00:00,  5.54s/it]

100%|███████████████████████████████████████████| 64/64 [05:39<00:00,  5.30s/it]

100%|███████████████████████████████████████████| 64/64 [05:54<00:00,  5.54s/it]

 72%|██████████████████████████████▉            | 46/64 [04:21<01:42,  5.68s/it]

KeyboardInterrupt



<Figure size 432x288 with 0 Axes>